# Working with parquet files

## Objective

+ In this assignment, we will use the data downloaded with the module `data_manager` to create features.

(11 pts total)

## Prerequisites

+ This notebook assumes that price data is available to you in the environment variable `PRICE_DATA`. If you have not done so, then execute the notebook `production_2_data_engineering.ipynb` to create this data set.


+ Load the environment variables using dotenv. (1 pt)

In [1]:
# Write your code below.
import os
from os.path import join, dirname
from dotenv import load_dotenv
env_path = join(os.path.abspath("./"), ".env")
load_dotenv(env_path)
env_path


'/home/stumac/code/uoft_dsi/scaling_to_production/02_activities/assignments/.env'

In [3]:
import dask
dask.config.set({'dataframe.query-planning': True})
import dask.dataframe as dd

+ Load the environment variable `PRICE_DATA`.
+ Use [glob](https://docs.python.org/3/library/glob.html) to find the path of all parquet files in the directory `PRICE_DATA`.

(1pt)

In [12]:
import os
from glob import glob

PRICE_DATA = os.getenv("PRICE_DATA")
print(PRICE_DATA)
parquet_files = glob(os.path.join(PRICE_DATA, "*/*/*.parquet"))
dd_px = dd.read_parquet(parquet_files).set_index("ticker")
dd_px.columns


../../05_src/data/prices/


Index(['Date', 'Open', 'High', 'Low', 'Close', 'Adj Close', 'Volume', 'sector',
       'subsector', 'year'],
      dtype='object')

For each ticker and using Dask, do the following:

+ Add lags for variables Close and Adj_Close.
+ Add returns based on Adjusted Close:
    
    - `returns`: (Adj Close / Adj Close_lag) - 1

+ Add the following range: 

    - `hi_lo_range`: this is the day's High minus Low.

+ Assign the result to `dd_feat`.

(4 pt)

In [13]:
# Write your code below.

import numpy as np
dd_feat = (dd_px.groupby('ticker', group_keys=False)
.apply(
      lambda x: x.assign(
        Close_lag_1=x['Close'].shift(1),
        Adj_Close_lag_1=x['Adj Close'].shift(1)
    )
)
.assign(
    returns = lambda x: x['Close']/x['Close_lag_1'] - 1
).assign(
    hi_low_range = lambda x: (x['High'] - x['Low'])
))


/tmp/ipykernel_432204/2586793623.py:4: UserWarning: `meta` is not specified, inferred from partial data. Please provide `meta` if the result is unexpected.
  Before: .apply(func)
  After:  .apply(func, meta={'x': 'f8', 'y': 'f8'}) for dataframe result
  or:     .apply(func, meta=('x', 'f8'))            for series result
  dd_feat = (dd_px.groupby('ticker', group_keys=False)


+ Convert the Dask data frame to a pandas data frame. 
+ Add a rolling average return calculation with a window of 10 days.
+ *Tip*: Consider using `.rolling(10).mean()`.

(3 pt)

In [15]:

df = dd_feat.compute()

df['rolling_mean'] = df['returns'].rolling(10).mean()
df


,Date,Open,High,Low,Close,Adj Close,Volume,sector,subsector,year,Close_lag_1,Adj_Close_lag_1,returns,hi_low_range,rolling_mean
ticker,,,,,,,,,,,,,,,
OXY,2008-01-02,74.767113,76.022171,74.019829,75.064110,47.411602,5664773,Energy,Oil & Gas Exploration & Production,2008,NaN,NaN,NaN,2.002342,NaN
OXY,2008-01-03,75.159920,77.440102,75.064110,77.133522,48.718674,6196471,Energy,Oil & Gas Exploration & Production,2008,75.064110,47.411602,0.027569,2.375992,NaN
OXY,2008-01-04,76.625755,76.884430,74.297661,74.498856,47.054588,6741635,Energy,Oil & Gas Exploration & Production,2008,77.133522,48.718674,-0.034157,2.586769,NaN
OXY,2008-01-07,74.728790,75.198242,71.567184,73.195892,46.231621,8300407,Energy,Oil & Gas Exploration & Production,2008,74.498856,47.054588,-0.017490,3.631058,NaN
OXY,2008-01-08,73.703667,74.518021,71.116898,71.375572,45.081890,8696833,Energy,Oil & Gas Exploration & Production,2008,73.195892,46.231621,-0.024869,3.401123,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
INTU,2012-12-24,61.139999,61.340000,60.740002,61.290001,55.384178,514500,Information Technology,Application Software,2012,61.439999,55.519718,-0.002441,0.599998,0.002818
INTU,2012-12-26,61.099998,61.490002,60.680000,60.869999,55.004642,743100,Information Technology,Application Software,2012,61.290001,55.384178,-0.006853,0.810001,0.000741
INTU,2012-12-27,60.840000,60.959999,59.630001,60.340000,54.525700,1510200,Information Technology,Application Software,2012,60.869999,55.004642,-0.008707,1.329998,0.000135


Please comment:

+ Was it necessary to convert to pandas to calculate the moving average return?

It's not necesary to convert to pandas before do the moving average. Dask can in fact handle it.
+ Would it have been better to do it in Dask? Why?

Possibly. Given the size of the dataset, it very well could be faster to use dask to apply a rolling mean.

(1 pt)

## Criteria

|Criteria|Complete|Incomplete|
|---------------------|----|----|
|Calculations         |Calculations were done correctly.|Calculations were not done correctly.|
|Explanation of answer|Answer was concise and explained the learner's reasoning in depth.|Answer was not concise and did not explained the learner's reasoning in depth.|

## Submission Information

🚨 **Please review our [Assignment Submission Guide](https://github.com/UofT-DSI/onboarding/blob/main/onboarding_documents/submissions.md)** 🚨 for detailed instructions on how to format, branch, and submit your work. Following these guidelines is crucial for your submissions to be evaluated correctly.

### Submission Parameters:
* Submission Due Date: `HH:MM AM/PM - DD/MM/YYYY`
* The branch name for your repo should be: `assignment-1`
* What to submit for this assignment:
    * This Jupyter Notebook (assignment_1.ipynb) should be populated and should be the only change in your pull request.
* What the pull request link should look like for this assignment: `https://github.com/<your_github_username>/production/pull/<pr_id>`
    * Open a private window in your browser. Copy and paste the link to your pull request into the address bar. Make sure you can see your pull request properly. This helps the technical facilitator and learning support staff review your submission easily.

Checklist:
- [ ] Created a branch with the correct naming convention.
- [ ] Ensured that the repository is public.
- [ ] Reviewed the PR description guidelines and adhered to them.
- [ ] Verify that the link is accessible in a private browser window.

If you encounter any difficulties or have questions, please don't hesitate to reach out to our team via our Slack at `#cohort-3-help`. Our Technical Facilitators and Learning Support staff are here to help you navigate any challenges.